# Задание 1
## Зависимости
Ubuntu/Debian

Python 2.7

Pyosmium и соответствующие зависимости:


In [ ]:
!sudo apt-get install build-essential libboost-python-dev libexpat1-dev zlib1g-dev libbz2-dev
!pip install osmium

Svgwrite:

In [ ]:
!pip install svgwrite

## Загрузка модулей и предобработка данных

In [1]:
import osmium as osm
import pandas as pd
import numpy as np
import svgwrite as svg

In [2]:
class Node:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon
        self.tags = {}
        self.count = 0
        
class Way:
    def __init__(self):
        self.tag = ""
        self.nodes = []
    

In [ ]:
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.nodes = {}
        self.ways = []
        
        # Рассчитаем информацию для отрисовки
        self.min_lat = 90
        self.max_lat = -90
        self.min_lon = 180
        self.max_lon = 0
        
    
    def node(self, n):
        x = Node(n.location.lat, n.location.lon)
        
        self.min_lat = min(self.min_lat, n.location.lat)
        self.max_lat = max(self.max_lat, n.location.lat)
        self.min_lon = min(self.min_lon, n.location.lon)
        self.max_lon = max(self.max_lon, n.location.lon)            
        
        for tag in n.tags:
            x.tags[tag.k] = tag.v   
        self.nodes[n.id] = x
        
    def way(self, w):
        proceed = False
        for tag in w.tags:
            if tag.k == "highway" and (tag.v == "primary" or tag.v == "secondary" or tag.v == "tertiary" or tag.v == "unclassified"):
                proceed  = True
                way_type = tag.v
        if proceed:
            x = Way()
            x.tag = way_type
            for nd in w.nodes:
                x.nodes.append(nd.ref)
                self.nodes[nd.ref].count+=1   #?
            self.ways.append(x)    

            
osmhandler = OSMHandler()
osmhandler.apply_file("Data/RU-SVE-raw.osm.pbf")
         
        

In [ ]:
#Удаляем промежуточные узлы в путях
for w in osmhandler.ways:
    w.shortened_nodes = []
    for i, x in enumerate(w.nodes):
        if osmhandler.nodes[x].count > 1 or i == 0 or i == len(w.nodes) - 1:
            w.shortened_nodes.append(x) 
        else:
            del osmhandler.nodes[x] 

In [ ]:
#Удаляем не встретившиеся узлы
for k in osmhandler.nodes.keys():
    if osmhandler.nodes[k].count == 0:
        del osmhandler.nodes[k]

##    Считаем матрицу смежности

In [6]:
node_count = len(osmhandler.nodes)
adjacency_matrix = np.zeros([node_count, node_count], dtype=np.int8) #int

for w in osmhandler.ways:
    for n in xrange(len(w.shortened_nodes) - 1):
        x = osmhandler.nodes.keys().index(w.shortened_nodes[n])
        y = osmhandler.nodes.keys().index(w.shortened_nodes[n + 1])
        adjacency_matrix[x,y] = 1
        adjacency_matrix[y,x] = 1

In [7]:
df_am = pd.DataFrame(adjacency_matrix, columns=osmhandler.nodes.keys())
df_am.index = osmhandler.nodes.keys()
df_am.head()

,4134012908,243796174,243796175,243796233,243796255,243796257,243796260,1470630250,243796371,775948804,...,360181861,360181864,360181866,360181868,176157987,314571441,3114061635,314571916,5165808816,229201118
4134012908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796175,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
243796233,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
243796255,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_am.to_csv('adjacency_matrix.csv')

## Рассчитываем список смежности

In [20]:
edge_list = []

for w in osmhandler.ways:
    for n in xrange(len(w.shortened_nodes) - 1):
        x = w.shortened_nodes[n]
        y = w.shortened_nodes[n + 1]
        edge_list.append([x,y])

In [11]:
# Сохраняем список ребер
node_list = [[x[0], osmhandler.nodes[x[0]].lat, osmhandler.nodes[x[0]].lon, x[1], osmhandler.nodes[x[1]].lat, osmhandler.nodes[x[1]].lon] for x in edge_list]

col_names = ['Node_1', 'lat', 'lon', 'Node_2', 'lat', 'lon']
df_el = pd.DataFrame(node_list, columns=col_names)
df_el.to_csv('extra/edge_list.csv')

In [12]:
adjacency_list = []

for n in osmhandler.nodes.keys():
    adjacent_nodes = []
    for e in edge_list:
        if e[0] == n:
            adjacent_nodes.append(e[1])
        if e[1] == n:
            adjacent_nodes.append(e[0])
    adjacency_list.append([n,adjacent_nodes])

df_al = pd.DataFrame(adjacency_list, columns=['Node', 'Adjacent nodes'])
df_al.to_csv('adjacency_list.csv', index=False)

## (Extra) Считаем матрицу инцидентности

In [13]:
incidence_matrix = np.zeros([node_count, len(edge_list)], dtype=np.int8)

for i,e in enumerate(edge_list):
    x = osmhandler.nodes.keys().index(e[0])
    y = osmhandler.nodes.keys().index(e[1])
    incidence_matrix[x, i] = 1
    incidence_matrix[y, i] = 1

In [14]:
edge_names = [str(x[0]) + "-" + str(x[1]) for x in edge_list]

df_im = pd.DataFrame(incidence_matrix, columns=edge_names)
df_im.index = osmhandler.nodes.keys()
df_im.head()

,175058331-492227867,416547560-416547601,179437086-175355780,175075315-175066119,175031590-241920422,175078842-175082606,175126202-710333758,175129445-710333742,175126202-175127331,176300884-1060972451,...,5170589322-411815606,1008892590-958472806,175031572-955659499,568024645-175031572,5228173768-175129449,3210491160-175124190,5253491622-725896225,5264049283-416547560,1608497388-2046596624,410049573-410049575
4134012908,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796174,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796175,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243796255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_im.to_csv('extra/incidence_matrix.csv')

## Рисуем SVG изображение

In [24]:
bot = osmhandler.min_lat
left = osmhandler.min_lon
up = osmhandler.max_lat
right = osmhandler.max_lon

scale = 1000

dwg = svg.Drawing('map.svg')
for e in edge_list:
    x1 = (osmhandler.nodes[e[0]].lat - bot)*scale
    y1 = (osmhandler.nodes[e[0]].lon - left)*scale
    x2 = (osmhandler.nodes[e[1]].lat - bot)*scale
    y2 = (osmhandler.nodes[e[1]].lon - left)*scale
    dwg.add(dwg.line((x1, y1), (x2, y2), stroke=svg.rgb(10, 10, 16, '%'), stroke_width=0.2))
dwg.save()